In [336]:
import sys
!{sys.executable} -m pip install openpyxl


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [339]:
import pandas as pd
import numpy as np
import openpyxl

# Load the Excel file
df = pd.read_excel((r'C:\Users\riyas\OneDrive - University of Birmingham\Documents\MSc Data Science\2. group project data science\natural disaster cleaning\natural disasters dataset.xlsx'), sheet_name="EM-DAT Data")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\riyas\\OneDrive - University of Birmingham\\Documents\\MSc Data Science\\2. group project data science\\natural disaster cleaning\\natural disasters dataset.xlsx'

In [ ]:
df.to_csv('natural_disasters_dataset.csv', index=False)

**Problems with the Dataset**
- Inconsistent Data
    - The dataset contains a mix of natural disasters and technological accidents, which might require separation depending on the analysis goals.

- Missing Values
    - There are likely many missing values represented by empty strings. 
    - These need to be handled (imputation or removal).

- Data Types
    - All columns are initially read as objects (strings). 
    - Many columns need to be converted to appropriate data types (numeric, datetime, etc.).

- Date Format
    - Date columns ('Entry Date', 'Last Update', Start Date, End Date) are in string format and need to be converted to datetime objects. 
    - The Start and End Date are split into Year, Month, and Day columns which should be merged into a single column.

- Magnitude
    - The 'Magnitude' and 'Magnitude Scale' columns need to be handled carefully. 
    - Missing magnitudes, different scales, and inconsistent entries need to be addressed.

- Location Data
    - Location data might need cleaning and standardization.

- Redundant/Irrelevant Columns
    - Some columns might be redundant or not useful for analysis (e.g., 'Admin Units'). 
    - These should be dropped to reduce dimensionality.

- Adjusted Damage Costs
    - There are columns for damage costs and adjusted damage costs, the adjusted columns may be more useful for comparing data across the years.

In [340]:
# Import Libraries and Load Data:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv(r'C:\Users\riyas\OneDrive - University of Birmingham\Documents\MSc Data Science\2. group project data science\natural disaster cleaning\natural_disasters_dataset.csv')

In [341]:
# Display initial info
print(df.head())
print(df.info())
print(df.describe()) # Descriptive statistics for numeric columns

          DisNo. Historic Classification Key Disaster Group  \
0  1900-0003-USA      Yes    nat-met-sto-tro        Natural   
1  1900-0005-USA      Yes    tec-ind-fir-fir  Technological   
2  1900-0006-JAM      Yes    nat-hyd-flo-flo        Natural   
3  1900-0007-JAM      Yes    nat-bio-epi-vir        Natural   
4  1900-0008-JPN      Yes    nat-geo-vol-ash        Natural   

     Disaster Subgroup      Disaster Type   Disaster Subtype External IDs  \
0       Meteorological              Storm   Tropical cyclone          NaN   
1  Industrial accident  Fire (Industrial)  Fire (Industrial)          NaN   
2         Hydrological              Flood    Flood (General)          NaN   
3           Biological           Epidemic      Viral disease          NaN   
4          Geophysical  Volcanic activity           Ash fall          NaN   

        Event Name  ISO  ... Reconstruction Costs ('000 US$)  \
0              NaN  USA  ...                             NaN   
1              NaN  USA  ...  

In [342]:
df

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,1900-0003-USA,Yes,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,USA,...,NaN,NaN,NaN,NaN,30000.0,1098720.0,2.730451,NaN,18/10/2004,17/10/2023
1,1900-0005-USA,Yes,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),NaN,NaN,USA,...,NaN,NaN,NaN,NaN,NaN,NaN,2.730451,NaN,01/07/2003,25/09/2023
2,1900-0006-JAM,Yes,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,JAM,...,NaN,NaN,NaN,NaN,NaN,NaN,2.730451,NaN,01/07/2003,25/09/2023
3,1900-0007-JAM,Yes,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,NaN,Gastroenteritis,JAM,...,NaN,NaN,NaN,NaN,NaN,NaN,2.730451,NaN,01/07/2003,25/09/2023
4,1900-0008-JPN,Yes,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,NaN,NaN,JPN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.730451,NaN,01/07/2003,25/09/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,VNM,...,NaN,NaN,NaN,NaN,10000.0,NaN,NaN,NaN,03/05/2024,12/08/2024
26910,2024-9338-NAM,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,GLIDE:DR-2024-000094,NaN,NAM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/06/2024,30/09/2024
26911,2024-9381-GRD,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,GLIDE:DR-2024-000079,NaN,GRD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14/06/2024,12/08/2024
26912,2024-9592-AGO,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,GLIDE:DR-2024-000095,NaN,AGO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/08/2024,12/08/2024


In [343]:
# Separate numerical and categorical columns
numerical_cols = df.select_dtypes(include=['number']).columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Print them separately
print("Numerical Columns:", numerical_cols)
print("\nCategorical Columns:", categorical_cols)

Numerical Columns: Index(['AID Contribution ('000 US$)', 'Magnitude', 'Latitude', 'Longitude',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No. Injured', 'No. Affected',
       'No. Homeless', 'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI'],
      dtype='object')

Categorical Columns: Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'Magnitude Scale', 'River Basin', 'Admin Units',
       'Entry Date', 'Last Update'],
      dtype='object')


In [344]:
# Drop irrelevant columns

columns_to_drop = ["Historic", "River Basin", "External IDs", "AID Contribution ('000 US$)", "Reconstruction Costs ('000 US$)", 
                   "Reconstruction Costs, Adjusted ('000 US$)", "Insured Damage ('000 US$)", "Insured Damage, Adjusted ('000 US$)", 
                   "Total Damage ('000 US$)", "Total Damage, Adjusted ('000 US$)", "CPI", "OFDA/BHA Response", "Appeal", "Declaration", "Total Deaths",
                   "No. Injured", "No. Affected", "No. Homeless", "Total Affected", "Entry Date", "Last Update"]
df = df.drop(columns=columns_to_drop)

In [ ]:
df

,DisNo.,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,ISO,Country,Subregion,...,Magnitude Scale,Latitude,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Admin Units
0,1900-0003-USA,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,NaN,USA,United States of America,Northern America,...,Kph,NaN,NaN,1900,9.0,8.0,1900,9.0,8.0,NaN
1,1900-0005-USA,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),NaN,USA,United States of America,Northern America,...,m3,NaN,NaN,1900,6.0,30.0,1900,6.0,30.0,NaN
2,1900-0006-JAM,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,JAM,Jamaica,Latin America and the Caribbean,...,Km2,NaN,NaN,1900,1.0,6.0,1900,1.0,6.0,NaN
3,1900-0007-JAM,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,Gastroenteritis,JAM,Jamaica,Latin America and the Caribbean,...,Vaccinated,NaN,NaN,1900,1.0,13.0,1900,1.0,13.0,NaN
4,1900-0008-JPN,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,NaN,JPN,Japan,Eastern Asia,...,NaN,NaN,NaN,1900,7.0,7.0,1900,7.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,VNM,Viet Nam,South-eastern Asia,...,Km2,NaN,NaN,2024,1.0,NaN,2024,6.0,NaN,NaN
26910,2024-9338-NAM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NAM,Namibia,Sub-Saharan Africa,...,Km2,NaN,NaN,2024,5.0,NaN,2024,9.0,NaN,NaN
26911,2024-9381-GRD,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,GRD,Grenada,Latin America and the Caribbean,...,Km2,NaN,NaN,2024,5.0,NaN,2024,7.0,NaN,NaN
26912,2024-9592-AGO,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,AGO,Angola,Sub-Saharan Africa,...,Km2,NaN,NaN,2024,6.0,NaN,2024,8.0,NaN,NaN


In [345]:
# Handle Missing Values:

# Identify columns with missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

Missing Values:
 Event Name          18456
Location             2493
Origin              22385
Associated Types    22917
Magnitude           21764
Magnitude Scale     10058
Latitude            24116
Longitude           24116
Start Month           504
Start Day            4055
End Month             812
End Day              3965
Admin Units         18429
dtype: int64


In [346]:
# Get columns with missing values
missing_cols = missing_values[missing_values > 0].index

# Separate categorical and numerical columns with missing values
cat_missing = [col for col in missing_cols if df[col].dtype == 'object']
num_missing = [col for col in missing_cols if df[col].dtype in ['int64', 'float64']]

# Print results
print("\nCategorical Columns with Missing Values:\n", cat_missing)
print("\nNumerical Columns with Missing Values:\n", num_missing)


Categorical Columns with Missing Values:
 ['Event Name', 'Location', 'Origin', 'Associated Types', 'Magnitude Scale', 'Admin Units']

Numerical Columns with Missing Values:
 ['Magnitude', 'Latitude', 'Longitude', 'Start Month', 'Start Day', 'End Month', 'End Day']


In [347]:
df[['Magnitude']]

,Magnitude
0,220.0
1,NaN
2,NaN
3,NaN
4,NaN
...,...
26909,NaN
26910,NaN
26911,NaN
26912,NaN


In [348]:
# Fill 'Magnitude' using median per disaster type
df['Magnitude'] = df.groupby('Disaster Type')['Magnitude'].transform(lambda x: x.fillna(x.median()))

df[['Magnitude']]

c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out,

,Magnitude
0,220.0
1,NaN
2,24680.0
3,200000.0
4,NaN
...,...
26909,3141.5
26910,3141.5
26911,3141.5
26912,3141.5


In [349]:
missing_magnitude = df['Magnitude'].isnull().sum()
print("Missing Magnitude Values:", missing_magnitude)

Missing Magnitude Values: 10556


In [350]:
df[['Latitude', 'Longitude']]

,Latitude,Longitude
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
26909,NaN,NaN
26910,NaN,NaN
26911,NaN,NaN
26912,NaN,NaN


In [351]:
df['Latitude'] = df.groupby('Disaster Type')['Latitude'].transform(lambda x: x.fillna(x.median()))
df['Longitude'] = df.groupby('Disaster Type')['Longitude'].transform(lambda x: x.fillna(x.median()))

df[['Latitude', 'Longitude']]

c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\riyas\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out,

,Latitude,Longitude
0,19.4328,104.125
1,NaN,NaN
2,15.1000,36.720
3,NaN,NaN
4,3.1700,22.102
...,...,...
26909,NaN,NaN
26910,NaN,NaN
26911,NaN,NaN
26912,NaN,NaN


In [352]:
missing_lat = df['Latitude'].isnull().sum()
missing_long = df['Longitude'].isnull().sum()
print("Missing Latitude Values:", missing_lat)
print("Missing Longitude Values:", missing_long)

Missing Latitude Values: 13257
Missing Longitude Values: 13257


In [353]:
# Check missing values across the columns
print(df[['Start Month', 'Start Day', 'End Month', 'End Day']].isnull().sum())


Start Month     504
Start Day      4055
End Month       812
End Day        3965
dtype: int64


In [354]:

# Get rows with missing values in any of these columns
missing_date_rows = df[df[['Start Month', 'Start Day', 'End Month', 'End Day']].isnull().any(axis=1)]
print(missing_date_rows)

              DisNo. Classification Key Disaster Group Disaster Subgroup  \
6      1900-9001-IND    nat-cli-dro-dro        Natural    Climatological   
7      1900-9002-CPV    nat-cli-dro-dro        Natural    Climatological   
8      1901-0001-UGA    nat-bio-epi-dis        Natural        Biological   
16     1902-0008-JPN    nat-geo-vol-ash        Natural       Geophysical   
22     1903-0002-USA    nat-hyd-flo-flo        Natural      Hydrological   
...              ...                ...            ...               ...   
26909  2024-9277-VNM    nat-cli-dro-dro        Natural    Climatological   
26910  2024-9338-NAM    nat-cli-dro-dro        Natural    Climatological   
26911  2024-9381-GRD    nat-cli-dro-dro        Natural    Climatological   
26912  2024-9592-AGO    nat-cli-dro-dro        Natural    Climatological   
26913  2024-9796-USA    nat-cli-dro-dro        Natural    Climatological   

           Disaster Type              Disaster Subtype Event Name  ISO  \
6            

In [355]:
# Forward-fill missing values (use previous valid entry)
df['Start Month'].fillna(method='ffill', inplace=True)
df['Start Day'].fillna(method='ffill', inplace=True)
df['End Month'].fillna(method='ffill', inplace=True)
df['End Day'].fillna(method='ffill', inplace=True)

df[['Start Month', 'Start Day', 'End Month', 'End Day']]




C:\Users\riyas\AppData\Local\Temp\ipykernel_26184\2099479049.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Start Month'].fillna(method='ffill', inplace=True)
C:\Users\riyas\AppData\Local\Temp\ipykernel_26184\2099479049.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Start Month'].fillna(method='ffill', inplace=True)
C:\Users\riyas\AppData\Local\Temp\ipykernel_26184\2099479049.py:3: FutureWarning: A value is trying to be s

,Start Month,Start Day,End Month,End Day
0,9.0,8.0,9.0,8.0
1,6.0,30.0,6.0,30.0
2,1.0,6.0,1.0,6.0
3,1.0,13.0,1.0,13.0
4,7.0,7.0,7.0,7.0
...,...,...,...,...
26909,1.0,1.0,6.0,31.0
26910,5.0,1.0,9.0,31.0
26911,5.0,1.0,7.0,31.0
26912,6.0,1.0,8.0,31.0


In [356]:
df[['Event Name', 'Location', 'Origin', 'Associated Types', 'Magnitude Scale']]

,Event Name,Location,Origin,Associated Types,Magnitude Scale
0,NaN,Galveston (Texas),NaN,"Avalanche (Snow, Debris)",Kph
1,NaN,"Hoboken, New York, Piers",NaN,Explosion,m3
2,NaN,Saint James,NaN,NaN,Km2
3,Gastroenteritis,Porus,NaN,NaN,Vaccinated
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
26909,NaN,"Can Duoc, Can Giuoc areas (Long An province); ...",NaN,NaN,Km2
26910,NaN,"Khomas, Ohangwena, Kavango West, Kunene, and K...",El Nino,Food shortage|Water shortage,Km2
26911,NaN,"Saint Andrew, Saint David, Saint George",NaN,Water shortage,Km2
26912,NaN,NaN,El Nino,Food shortage,Km2


In [357]:
# Fill missing values in categorical columns with 'Unknown' or a placeholder

df['Event Name'].fillna('Unknown', inplace=True)
df['Location'].fillna('Unknown', inplace=True)
df['Origin'].fillna('Unknown', inplace=True)
df['Associated Types'].fillna('Unknown', inplace=True)
df['Magnitude Scale'].fillna('Unknown', inplace=True)

df[['Event Name', 'Location', 'Origin', 'Associated Types', 'Magnitude Scale']]


C:\Users\riyas\AppData\Local\Temp\ipykernel_26184\199430895.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Event Name'].fillna('Unknown', inplace=True)
C:\Users\riyas\AppData\Local\Temp\ipykernel_26184\199430895.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,Event Name,Location,Origin,Associated Types,Magnitude Scale
0,Unknown,Galveston (Texas),Unknown,"Avalanche (Snow, Debris)",Kph
1,Unknown,"Hoboken, New York, Piers",Unknown,Explosion,m3
2,Unknown,Saint James,Unknown,Unknown,Km2
3,Gastroenteritis,Porus,Unknown,Unknown,Vaccinated
4,Unknown,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...
26909,Unknown,"Can Duoc, Can Giuoc areas (Long An province); ...",Unknown,Unknown,Km2
26910,Unknown,"Khomas, Ohangwena, Kavango West, Kunene, and K...",El Nino,Food shortage|Water shortage,Km2
26911,Unknown,"Saint Andrew, Saint David, Saint George",Unknown,Water shortage,Km2
26912,Unknown,Unknown,El Nino,Food shortage,Km2


In [358]:
df['Start Date'] = df[['Start Day', 'Start Month', 'Start Year']].astype(str).agg('/'.join, axis=1)
df['End Date'] = df[['End Day', 'End Month', 'End Year']].astype(str).agg('/'.join, axis=1)

# Check if the strings are being generated correctly
df[['Start Date', 'End Date']]


,Start Date,End Date
0,8.0/9.0/1900,8.0/9.0/1900
1,30.0/6.0/1900,30.0/6.0/1900
2,6.0/1.0/1900,6.0/1.0/1900
3,13.0/1.0/1900,13.0/1.0/1900
4,7.0/7.0/1900,7.0/7.0/1900
...,...,...
26909,1.0/1.0/2024,31.0/6.0/2024
26910,1.0/5.0/2024,31.0/9.0/2024
26911,1.0/5.0/2024,31.0/7.0/2024
26912,1.0/6.0/2024,31.0/8.0/2024


In [359]:
# Convert numeric values to integers (and remove decimals)
df['Start Day'] = df['Start Day'].astype(int)
df['Start Month'] = df['Start Month'].astype(int)
df['Start Year'] = df['Start Year'].astype(int)

df['End Day'] = df['End Day'].astype(int)
df['End Month'] = df['End Month'].astype(int)
df['End Year'] = df['End Year'].astype(int)

# Format the date correctly by ensuring day and month are 2 digits (e.g., 01, 09)
df['Start Date'] = df[['Start Day', 'Start Month', 'Start Year']].apply(lambda x: f"{x[0]:02d}/{x[1]:02d}/{x[2]}", axis=1)
df['End Date'] = df[['End Day', 'End Month', 'End Year']].apply(lambda x: f"{x[0]:02d}/{x[1]:02d}/{x[2]}", axis=1)

# Show the resulting 'Start Date' and 'End Date' columns
df[['Start Date', 'End Date']]


C:\Users\riyas\AppData\Local\Temp\ipykernel_26184\3385512483.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['Start Date'] = df[['Start Day', 'Start Month', 'Start Year']].apply(lambda x: f"{x[0]:02d}/{x[1]:02d}/{x[2]}", axis=1)
C:\Users\riyas\AppData\Local\Temp\ipykernel_26184\3385512483.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['End Date'] = df[['End Day', 'End Month', 'End Year']].apply(lambda x: f"{x[0]:02d}/{x[1]:02d}/{x[2]}", axis=1)


,Start Date,End Date
0,08/09/1900,08/09/1900
1,30/06/1900,30/06/1900
2,06/01/1900,06/01/1900
3,13/01/1900,13/01/1900
4,07/07/1900,07/07/1900
...,...,...
26909,01/01/2024,31/06/2024
26910,01/05/2024,31/09/2024
26911,01/05/2024,31/07/2024
26912,01/06/2024,31/08/2024


In [360]:
df

,DisNo.,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,ISO,Country,Subregion,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Admin Units,Start Date,End Date
0,1900-0003-USA,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,Unknown,USA,United States of America,Northern America,...,104.125,1900,9,8,1900,9,8,NaN,08/09/1900,08/09/1900
1,1900-0005-USA,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),Unknown,USA,United States of America,Northern America,...,NaN,1900,6,30,1900,6,30,NaN,30/06/1900,30/06/1900
2,1900-0006-JAM,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),Unknown,JAM,Jamaica,Latin America and the Caribbean,...,36.720,1900,1,6,1900,1,6,NaN,06/01/1900,06/01/1900
3,1900-0007-JAM,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,Gastroenteritis,JAM,Jamaica,Latin America and the Caribbean,...,NaN,1900,1,13,1900,1,13,NaN,13/01/1900,13/01/1900
4,1900-0008-JPN,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,Unknown,JPN,Japan,Eastern Asia,...,22.102,1900,7,7,1900,7,7,NaN,07/07/1900,07/07/1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,VNM,Viet Nam,South-eastern Asia,...,NaN,2024,1,1,2024,6,31,NaN,01/01/2024,31/06/2024
26910,2024-9338-NAM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,NAM,Namibia,Sub-Saharan Africa,...,NaN,2024,5,1,2024,9,31,NaN,01/05/2024,31/09/2024
26911,2024-9381-GRD,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,GRD,Grenada,Latin America and the Caribbean,...,NaN,2024,5,1,2024,7,31,NaN,01/05/2024,31/07/2024
26912,2024-9592-AGO,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,AGO,Angola,Sub-Saharan Africa,...,NaN,2024,6,1,2024,8,31,NaN,01/06/2024,31/08/2024


In [361]:
df = df.drop(columns=['Start Day', 'Start Month', 'Start Year', 'End Day', 'End Month', 'End Year', 'Admin Units'])
df

,DisNo.,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,ISO,Country,Subregion,Region,Location,Origin,Associated Types,Magnitude,Magnitude Scale,Latitude,Longitude,Start Date,End Date
0,1900-0003-USA,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,Unknown,USA,United States of America,Northern America,Americas,Galveston (Texas),Unknown,"Avalanche (Snow, Debris)",220.0,Kph,19.4328,104.125,08/09/1900,08/09/1900
1,1900-0005-USA,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),Unknown,USA,United States of America,Northern America,Americas,"Hoboken, New York, Piers",Unknown,Explosion,NaN,m3,NaN,NaN,30/06/1900,30/06/1900
2,1900-0006-JAM,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),Unknown,JAM,Jamaica,Latin America and the Caribbean,Americas,Saint James,Unknown,Unknown,24680.0,Km2,15.1000,36.720,06/01/1900,06/01/1900
3,1900-0007-JAM,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,Gastroenteritis,JAM,Jamaica,Latin America and the Caribbean,Americas,Porus,Unknown,Unknown,200000.0,Vaccinated,NaN,NaN,13/01/1900,13/01/1900
4,1900-0008-JPN,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,Unknown,JPN,Japan,Eastern Asia,Asia,Unknown,Unknown,Unknown,NaN,Unknown,3.1700,22.102,07/07/1900,07/07/1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,VNM,Viet Nam,South-eastern Asia,Asia,"Can Duoc, Can Giuoc areas (Long An province); ...",Unknown,Unknown,3141.5,Km2,NaN,NaN,01/01/2024,31/06/2024
26910,2024-9338-NAM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,NAM,Namibia,Sub-Saharan Africa,Africa,"Khomas, Ohangwena, Kavango West, Kunene, and K...",El Nino,Food shortage|Water shortage,3141.5,Km2,NaN,NaN,01/05/2024,31/09/2024
26911,2024-9381-GRD,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,GRD,Grenada,Latin America and the Caribbean,Americas,"Saint Andrew, Saint David, Saint George",Unknown,Water shortage,3141.5,Km2,NaN,NaN,01/05/2024,31/07/2024
26912,2024-9592-AGO,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,AGO,Angola,Sub-Saharan Africa,Africa,Unknown,El Nino,Food shortage,3141.5,Km2,NaN,NaN,01/06/2024,31/08/2024


In [363]:
# Convert magnitude to numeric
df['Magnitude'] = pd.to_numeric(df['Magnitude'], errors='coerce')

df[['Magnitude', 'Magnitude Scale']]

,Magnitude,Magnitude Scale
0,220.0,Kph
1,NaN,m3
2,24680.0,Km2
3,200000.0,Vaccinated
4,NaN,Unknown
...,...,...
26909,3141.5,Km2
26910,3141.5,Km2
26911,3141.5,Km2
26912,3141.5,Km2


In [366]:
df = df.drop(columns=['Magnitude Scale', 'Magnitude'])
df

,DisNo.,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,ISO,Country,Subregion,Region,Location,Origin,Associated Types,Latitude,Longitude,Start Date,End Date
0,1900-0003-USA,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,Unknown,USA,United States of America,Northern America,Americas,Galveston (Texas),Unknown,"Avalanche (Snow, Debris)",19.4328,104.125,08/09/1900,08/09/1900
1,1900-0005-USA,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),Unknown,USA,United States of America,Northern America,Americas,"Hoboken, New York, Piers",Unknown,Explosion,NaN,NaN,30/06/1900,30/06/1900
2,1900-0006-JAM,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),Unknown,JAM,Jamaica,Latin America and the Caribbean,Americas,Saint James,Unknown,Unknown,15.1000,36.720,06/01/1900,06/01/1900
3,1900-0007-JAM,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,Gastroenteritis,JAM,Jamaica,Latin America and the Caribbean,Americas,Porus,Unknown,Unknown,NaN,NaN,13/01/1900,13/01/1900
4,1900-0008-JPN,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,Unknown,JPN,Japan,Eastern Asia,Asia,Unknown,Unknown,Unknown,3.1700,22.102,07/07/1900,07/07/1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,VNM,Viet Nam,South-eastern Asia,Asia,"Can Duoc, Can Giuoc areas (Long An province); ...",Unknown,Unknown,NaN,NaN,01/01/2024,31/06/2024
26910,2024-9338-NAM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,NAM,Namibia,Sub-Saharan Africa,Africa,"Khomas, Ohangwena, Kavango West, Kunene, and K...",El Nino,Food shortage|Water shortage,NaN,NaN,01/05/2024,31/09/2024
26911,2024-9381-GRD,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,GRD,Grenada,Latin America and the Caribbean,Americas,"Saint Andrew, Saint David, Saint George",Unknown,Water shortage,NaN,NaN,01/05/2024,31/07/2024
26912,2024-9592-AGO,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,AGO,Angola,Sub-Saharan Africa,Africa,Unknown,El Nino,Food shortage,NaN,NaN,01/06/2024,31/08/2024


In [367]:
# Handle Missing Values:

# Identify columns with missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

Missing Values:
 Latitude     13257
Longitude    13257
dtype: int64


In [368]:
# Save the preprocessed DataFrame to a new CSV file
df.to_csv('draft2_cleaned_natural_disasters_dataset.csv', index=False)


**Add Missing Longitude and Latitudes by Country Centre Point**

In [527]:
# Import Libraries and Load Data:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv(r'C:\Users\riyas\OneDrive - University of Birmingham\Documents\MSc Data Science\2. group project data science\natural disaster cleaning\draft2_cleaned_natural_disasters_dataset.csv')

In [528]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [529]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

In [530]:
# Initialize geolocator
geolocator = Nominatim(user_agent="geoapiExercises")

In [531]:
country_list = df['Country'].unique()
print(country_list)

['United States of America' 'Jamaica' 'Japan' 'Türkiye' 'India'
 'Cabo Verde' 'Uganda' 'Belgium' 'Guatemala' 'Myanmar' 'Martinique'
 'Saint Vincent and the Grenadines' 'Soviet Union' 'China' 'Canada'
 'France' 'Comoros' 'Israel' 'Iran (Islamic Republic of)' 'Niger'
 'Bangladesh' 'Taiwan (Province of China)' 'Greece' 'Italy' 'Philippines'
 'Albania' 'Colombia' 'Chile'
 'China, Hong Kong Special Administrative Region'
 'United Kingdom of Great Britain and Northern Ireland' 'Germany'
 'Romania' 'Indonesia' 'Mexico' 'Haiti' 'Morocco' 'Portugal' 'Pakistan'
 'Costa Rica' 'Algeria' 'Burkina Faso' 'Gambia' 'Guinea-Bissau' 'Mali'
 'Mauritania' 'Senegal' 'Chad' 'Kazakhstan' 'Spain' 'Peru' 'Tokelau'
 'Belarus' 'Bulgaria' 'Netherlands (Kingdom of the)' 'Denmark'
 'Puerto Rico' 'New Zealand' 'Anguilla' 'Germany Federal Republic'
 'Argentina' 'Ecuador' 'Bahamas' 'Cuba' 'Egypt' 'Jordan' 'Poland'
 'Guadeloupe' 'Montserrat' 'Saint Kitts and Nevis' 'Brazil'
 'Dominican Republic' 'Czechoslovakia' 'Nicara

In [533]:
# Filter the rows where either longitude or latitude is NaN
countries_with_missing_coordinates = df[df['Longitude'].isna() & df['Latitude'].isna()]

# Get the unique list of countries with missing values in both longitude and latitude
countries_with_missing_coords_list = countries_with_missing_coordinates['Country'].unique()

# Convert to list and print the result
print(countries_with_missing_coords_list)


['United States of America' 'Jamaica' 'India' 'Cabo Verde' 'Uganda'
 'Belgium' 'Canada' 'France' 'Niger'
 'United Kingdom of Great Britain and Northern Ireland' 'Germany' 'China'
 'Indonesia' 'Mexico' 'Burkina Faso' 'Gambia' 'Guinea-Bissau' 'Mali'
 'Mauritania' 'Senegal' 'Chad' 'Spain' 'Japan' 'Belarus' 'Bulgaria'
 'Soviet Union' 'Netherlands (Kingdom of the)' 'Denmark' 'Bangladesh'
 'New Zealand' 'Anguilla' 'Türkiye' 'Brazil' 'Czechoslovakia' 'Australia'
 'Germany Federal Republic' 'Colombia' 'Switzerland' 'Honduras' 'Italy'
 'Egypt' 'Bolivia (Plurinational State of)' 'Algeria' 'Romania'
 'Philippines' 'Venezuela (Bolivarian Republic of)' 'Sudan' 'South Africa'
 'Hungary' 'Puerto Rico' 'Portugal' 'Peru' 'Ireland' 'Sweden' 'Greece'
 'Libya' 'Cuba' 'Norway' 'China, Hong Kong Special Administrative Region'
 'Democratic Republic of the Congo' 'Congo' 'Poland'
 'German Democratic Republic' 'Republic of Korea' 'Pakistan'
 'Iran (Islamic Republic of)' 'Finland' 'Morocco' 'Guatemala' 'Chile'


In [535]:
import sys
!{sys.executable} -m pip install geopandas


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [536]:
import geopandas as gpd
import pandas as pd

In [537]:
# Load your dataset (natural disasters)
df_natural_disasters = pd.read_csv(r"C:\Users\riyas\OneDrive - University of Birmingham\Documents\MSc Data Science\2. group project data science\natural disaster cleaning\draft2_cleaned_natural_disasters_dataset.csv")  

# Load country coordinates dataset
df_country_centroids = pd.read_csv(r"C:\Users\riyas\OneDrive - University of Birmingham\Documents\MSc Data Science\2. group project data science\natural disaster cleaning\countries_centroids.csv")  

df_country_centroids = df_country_centroids.drop(columns=['COUNTRYAFF', 'ISO', 'AFF_ISO'])
df_country_centroids.rename(columns={"COUNTRY": "Country"}, inplace=True)
df_country_centroids.rename(columns={"longitude": "Longitude"}, inplace=True)
df_country_centroids.rename(columns={"latitude": "Latitude"}, inplace=True)
df_country_centroids["Country"] = df_country_centroids["Country"].str.strip()


# Check the structure of the coordinates dataset
df_country_centroids

,Longitude,Latitude,Country
0,-170.700732,-14.305712,American Samoa
1,166.638003,19.302046,United States Minor Outlying Islands
2,-159.787689,-21.222613,Cook Islands
3,-149.400417,-17.674684,French Polynesia
4,-169.868781,-19.052309,Niue
...,...,...,...
244,145.741197,15.178064,Northern Mariana Islands
245,134.579651,7.534776,Palau
246,98.670499,59.039434,Russian Federation
247,-3.651625,40.365008,Spain


['United States of America' 'Jamaica' 'India' 'Cabo Verde' 'Uganda'
 'Belgium' 'Canada' 'France' 'Niger'
 'United Kingdom of Great Britain and Northern Ireland' 'Germany' 'China'
 'Indonesia' 'Mexico' 'Burkina Faso' 'Gambia' 'Guinea-Bissau' 'Mali'
 'Mauritania' 'Senegal' 'Chad' 'Spain' 'Japan' 'Belarus' 'Bulgaria'
 'Soviet Union' 'Netherlands (Kingdom of the)' 'Denmark' 'Bangladesh'
 'New Zealand' 'Anguilla' 'Türkiye' 'Brazil' 'Czechoslovakia' 'Australia'
 'Germany Federal Republic' 'Colombia' 'Switzerland' 'Honduras' 'Italy'
 'Egypt' 'Bolivia (Plurinational State of)' 'Algeria' 'Romania'
 'Philippines' 'Venezuela (Bolivarian Republic of)' 'Sudan' 'South Africa'
 'Hungary' 'Puerto Rico' 'Portugal' 'Peru' 'Ireland' 'Sweden' 'Greece'
 'Libya' 'Cuba' 'Norway' 'China, Hong Kong Special Administrative Region'
 'Democratic Republic of the Congo' 'Congo' 'Poland'
 'German Democratic Republic' 'Republic of Korea' 'Pakistan'
 'Iran (Islamic Republic of)' 'Finland' 'Morocco' 'Guatemala' 'Chile'
 'Myanmar' 'Nepal' 'Haiti' 'Paraguay' 'Viet Nam' 'Afghanistan'
 'Madagascar' 'Panama' 'Ecuador' 'Somalia' 'Dominican Republic' 'Malawi'
 'Yugoslavia' 'Bahamas' 'Togo' 'Syrian Arab Republic' 'Botswana'
 'Ethiopia' 'Kenya' 'Saudi Arabia' 'Sri Lanka' 'Nicaragua' 'Malaysia'
 'United Republic of Tanzania' 'Thailand' 'Argentina' 'Guadeloupe'
 'Lesotho' 'El Salvador' 'Nigeria' 'Benin' 'Iraq' 'Yemen Arab Republic'
 'Cyprus' 'Zambia' 'Côte d’Ivoire' 'United Arab Emirates' 'Uruguay'
 'Cameroon' 'Ghana' 'Canary Islands' 'Costa Rica' 'Cambodia'
 "People's Democratic Republic of Yemen"
 'China, Macao Special Administrative Region' 'Rwanda' 'Mozambique'
 'Kiribati' 'Liberia' 'Tunisia' "Lao People's Democratic Republic"
 'Maldives' 'Guyana' 'Burundi' 'Bahrain' 'Djibouti' 'Trinidad and Tobago'
 'Taiwan (Province of China)' 'Mauritius' 'Guinea' 'Papua New Guinea'
 'Jordan' 'Comoros' 'Angola' 'Namibia' 'Eswatini' 'Zimbabwe' 'Belize'
 'State of Palestine' 'Samoa' 'Sierra Leone' 'Israel' 'Lebanon' 'Fiji'
 'Central African Republic' 'Antigua and Barbuda' 'Sao Tome and Principe'
 'Bhutan' 'Albania' 'Mongolia' 'Singapore' 'Equatorial Guinea'
 'Azores Islands' 'Bermuda' "Democratic People's Republic of Korea"
 'Gabon' 'Suriname' 'Austria' 'Vanuatu' 'Grenada' 'Barbados' 'Georgia'
 'Cook Islands' 'Russian Federation' 'Malta' 'Réunion' 'Solomon Islands'
 'Ukraine' 'Azerbaijan' 'Tajikistan' 'Croatia' 'Armenia' 'Yemen'
 'Uzbekistan' 'Lithuania' 'North Macedonia' 'Eritrea' 'Kazakhstan'
 'Estonia' 'Czechia' 'Serbia Montenegro' 'Kyrgyzstan' 'Slovenia'
 'Slovakia' 'Seychelles' 'Palau' 'Guam' 'Bosnia and Herzegovina'
 'Brunei Darussalam' 'Turks and Caicos Islands' 'Dominica' 'Turkmenistan'
 'Micronesia (Federated States of)' 'Republic of Moldova' 'Niue' 'Tuvalu'
 'Latvia' 'Marshall Islands' 'Netherlands Antilles' 'Oman' 'Luxembourg'
 'New Caledonia' 'Timor-Leste' 'Serbia' 'French Polynesia' 'Mayotte'
 'Qatar' 'Tonga' 'Kuwait' 'South Sudan' 'Martinique' 'Saint Lucia'
 'Saint Vincent and the Grenadines' 'Montenegro' 'Curaçao' 'Iceland'
 'Liechtenstein']

In [538]:
df_natural_disasters["Country"] = df_natural_disasters["Country"].str.strip()

df_natural_disasters

,DisNo.,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,ISO,Country,Subregion,Region,Location,Origin,Associated Types,Latitude,Longitude,Start Date,End Date
0,1900-0003-USA,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,Unknown,USA,United States of America,Northern America,Americas,Galveston (Texas),Unknown,"Avalanche (Snow, Debris)",19.4328,104.125,08/09/1900,08/09/1900
1,1900-0005-USA,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),Unknown,USA,United States of America,Northern America,Americas,"Hoboken, New York, Piers",Unknown,Explosion,NaN,NaN,30/06/1900,30/06/1900
2,1900-0006-JAM,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),Unknown,JAM,Jamaica,Latin America and the Caribbean,Americas,Saint James,Unknown,Unknown,15.1000,36.720,06/01/1900,06/01/1900
3,1900-0007-JAM,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,Gastroenteritis,JAM,Jamaica,Latin America and the Caribbean,Americas,Porus,Unknown,Unknown,NaN,NaN,13/01/1900,13/01/1900
4,1900-0008-JPN,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,Unknown,JPN,Japan,Eastern Asia,Asia,Unknown,Unknown,Unknown,3.1700,22.102,07/07/1900,07/07/1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,VNM,Viet Nam,South-eastern Asia,Asia,"Can Duoc, Can Giuoc areas (Long An province); ...",Unknown,Unknown,NaN,NaN,01/01/2024,31/06/2024
26910,2024-9338-NAM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,NAM,Namibia,Sub-Saharan Africa,Africa,"Khomas, Ohangwena, Kavango West, Kunene, and K...",El Nino,Food shortage|Water shortage,NaN,NaN,01/05/2024,31/09/2024
26911,2024-9381-GRD,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,GRD,Grenada,Latin America and the Caribbean,Americas,"Saint Andrew, Saint David, Saint George",Unknown,Water shortage,NaN,NaN,01/05/2024,31/07/2024
26912,2024-9592-AGO,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,AGO,Angola,Sub-Saharan Africa,Africa,Unknown,El Nino,Food shortage,NaN,NaN,01/06/2024,31/08/2024


In [539]:
print(df_natural_disasters.columns)  # Check column names in disasters dataset
print(df_country_centroids.columns)  # Check column names in centroids dataset

Index(['DisNo.', 'Classification Key', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Event Name', 'ISO', 'Country',
       'Subregion', 'Region', 'Location', 'Origin', 'Associated Types',
       'Latitude', 'Longitude', 'Start Date', 'End Date'],
      dtype='object')
Index(['Longitude', 'Latitude', 'Country'], dtype='object')


In [540]:
print(df_natural_disasters["Country"].unique())  # Unique country names in disasters dataset
print(df_country_centroids["Country"].unique())  # Unique country names in centroids dataset


['United States of America' 'Jamaica' 'Japan' 'Türkiye' 'India'
 'Cabo Verde' 'Uganda' 'Belgium' 'Guatemala' 'Myanmar' 'Martinique'
 'Saint Vincent and the Grenadines' 'Soviet Union' 'China' 'Canada'
 'France' 'Comoros' 'Israel' 'Iran (Islamic Republic of)' 'Niger'
 'Bangladesh' 'Taiwan (Province of China)' 'Greece' 'Italy' 'Philippines'
 'Albania' 'Colombia' 'Chile'
 'China, Hong Kong Special Administrative Region'
 'United Kingdom of Great Britain and Northern Ireland' 'Germany'
 'Romania' 'Indonesia' 'Mexico' 'Haiti' 'Morocco' 'Portugal' 'Pakistan'
 'Costa Rica' 'Algeria' 'Burkina Faso' 'Gambia' 'Guinea-Bissau' 'Mali'
 'Mauritania' 'Senegal' 'Chad' 'Kazakhstan' 'Spain' 'Peru' 'Tokelau'
 'Belarus' 'Bulgaria' 'Netherlands (Kingdom of the)' 'Denmark'
 'Puerto Rico' 'New Zealand' 'Anguilla' 'Germany Federal Republic'
 'Argentina' 'Ecuador' 'Bahamas' 'Cuba' 'Egypt' 'Jordan' 'Poland'
 'Guadeloupe' 'Montserrat' 'Saint Kitts and Nevis' 'Brazil'
 'Dominican Republic' 'Czechoslovakia' 'Nicara

In [542]:
print(set(df_natural_disasters["Country"]) - set(df_country_centroids["Country"]))

{'Democratic Republic of the Congo', 'Iran (Islamic Republic of)', 'Bolivia (Plurinational State of)', 'German Democratic Republic', 'Serbia Montenegro', 'Viet Nam', 'United States Virgin Islands', 'Germany Federal Republic', 'China, Macao Special Administrative Region', 'Canary Islands', 'Saint Martin (French Part)', 'Netherlands (Kingdom of the)', 'Republic of Moldova', 'Soviet Union', 'China, Hong Kong Special Administrative Region', 'Micronesia (Federated States of)', 'Türkiye', 'United States of America', 'Azores Islands', 'Republic of Korea', 'Czechoslovakia', 'Yemen Arab Republic', 'Czechia', 'Sint Maarten (Dutch part)', 'State of Palestine', 'Syrian Arab Republic', 'United Republic of Tanzania', 'Curaçao', "Democratic People's Republic of Korea", 'United Kingdom of Great Britain and Northern Ireland', 'Venezuela (Bolivarian Republic of)', 'Taiwan (Province of China)', "People's Democratic Republic of Yemen", 'Wallis and Futuna Islands', 'Côte d’Ivoire', "Lao People's Democratic

In [543]:
from thefuzz import process

# Function to match country names
def match_country(name, choices):
    match, score = process.extractOne(name, choices)
    return match if score > 85 else name  # Adjust threshold if needed

# Apply fuzzy matching
df_natural_disasters["Country"] = df_natural_disasters["Country"].apply(
    lambda x: match_country(x, df_country_centroids["Country"].unique().tolist())
)


In [546]:
# Filter the rows where either longitude or latitude is NaN
missing_coordinates = df_merged[df_merged['Longitude'].isna() & df_merged['Latitude'].isna()]

# Get the unique list of countries with missing values in both longitude and latitude
missing_coords_list = missing_coordinates['Country'].unique()

# Convert to list and print the result
print(missing_coords_list)

['Soviet Union' 'Türkiye' 'China, Hong Kong Special Administrative Region'
 'Yugoslavia' 'China, Macao Special Administrative Region' 'Czechia']


In [547]:
df_merged = df_natural_disasters.merge(
    df_country_centroids, how="left", on="Country", suffixes=("", "_centroid")
)

# Fill missing values from centroids
df_merged["Longitude"] = df_merged["Longitude"].fillna(df_merged["Longitude_centroid"])
df_merged["Latitude"] = df_merged["Latitude"].fillna(df_merged["Latitude_centroid"])

# Drop unnecessary centroid columns
df_merged.drop(columns=["Longitude_centroid", "Latitude_centroid"], inplace=True)

# Verify if missing values remain
print(df_merged.isna().sum())

df_merged



DisNo.                  0
Classification Key      0
Disaster Group          0
Disaster Subgroup       0
Disaster Type           0
Disaster Subtype        0
Event Name              0
ISO                     0
Country                 0
Subregion               0
Region                  0
Location                0
Origin                  0
Associated Types        0
Latitude              322
Longitude             322
Start Date              0
End Date                0
dtype: int64


,DisNo.,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,ISO,Country,Subregion,Region,Location,Origin,Associated Types,Latitude,Longitude,Start Date,End Date
0,1900-0003-USA,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,Unknown,USA,United States,Northern America,Americas,Galveston (Texas),Unknown,"Avalanche (Snow, Debris)",19.432800,104.125000,08/09/1900,08/09/1900
1,1900-0005-USA,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),Unknown,USA,United States,Northern America,Americas,"Hoboken, New York, Piers",Unknown,Explosion,38.820809,-96.331617,30/06/1900,30/06/1900
2,1900-0006-JAM,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),Unknown,JAM,Jamaica,Latin America and the Caribbean,Americas,Saint James,Unknown,Unknown,15.100000,36.720000,06/01/1900,06/01/1900
3,1900-0007-JAM,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,Gastroenteritis,JAM,Jamaica,Latin America and the Caribbean,Americas,Porus,Unknown,Unknown,18.122079,-77.303589,13/01/1900,13/01/1900
4,1900-0008-JPN,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,Unknown,JPN,Japan,Eastern Asia,Asia,Unknown,Unknown,Unknown,3.170000,22.102000,07/07/1900,07/07/1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,VNM,Vietnam,South-eastern Asia,Asia,"Can Duoc, Can Giuoc areas (Long An province); ...",Unknown,Unknown,16.517347,105.913388,01/01/2024,31/06/2024
26910,2024-9338-NAM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,NAM,Namibia,Sub-Saharan Africa,Africa,"Khomas, Ohangwena, Kavango West, Kunene, and K...",El Nino,Food shortage|Water shortage,-21.908582,18.164513,01/05/2024,31/09/2024
26911,2024-9381-GRD,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,GRD,Grenada,Latin America and the Caribbean,Americas,"Saint Andrew, Saint David, Saint George",Unknown,Water shortage,12.112927,-61.679379,01/05/2024,31/07/2024
26912,2024-9592-AGO,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,AGO,Angola,Sub-Saharan Africa,Africa,Unknown,El Nino,Food shortage,-12.167424,17.651769,01/06/2024,31/08/2024


In [548]:
# Fill missing Latitudes and Longitudes using Country's centroid

# Create a dictionary of country centroids (latitude and longitude)
dict_country_centroids = {
    'Soviet Union': (59.35562, 89.92896),
    'Türkiye': (38.93207363123396, 35.56886764076691),
    'China, Hong Kong Special Administrative Region': (22.302711, 114.177216),
    'Yugoslavia': (43.849998, 18.25),
    'China, Macao Special Administrative Region': (22.210928, 113.552971),
    'Czechia': (50.073658, 14.418540)
    }

# Function to fill missing lat/lon based on country
def fill_lat_lon(row):
    if pd.isna(row['Latitude']) and pd.isna(row['Longitude']):  
        return dict_country_centroids.get(row['Country'], (np.nan, np.nan))  # Get centroid or NaN
    return row['Latitude'], row['Longitude']  # Keep original if not missing

# Apply the function to fill missing values
df_merged[['Latitude', 'Longitude']] = df_merged.apply(fill_lat_lon, axis=1, result_type="expand")

# Verify that missing values are handled (check Latitude and Longitude)
print(df_merged.isnull().sum())

DisNo.                0
Classification Key    0
Disaster Group        0
Disaster Subgroup     0
Disaster Type         0
Disaster Subtype      0
Event Name            0
ISO                   0
Country               0
Subregion             0
Region                0
Location              0
Origin                0
Associated Types      0
Latitude              0
Longitude             0
Start Date            0
End Date              0
dtype: int64


In [549]:
df_merged

,DisNo.,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,ISO,Country,Subregion,Region,Location,Origin,Associated Types,Latitude,Longitude,Start Date,End Date
0,1900-0003-USA,nat-met-sto-tro,Natural,Meteorological,Storm,Tropical cyclone,Unknown,USA,United States,Northern America,Americas,Galveston (Texas),Unknown,"Avalanche (Snow, Debris)",19.432800,104.125000,08/09/1900,08/09/1900
1,1900-0005-USA,tec-ind-fir-fir,Technological,Industrial accident,Fire (Industrial),Fire (Industrial),Unknown,USA,United States,Northern America,Americas,"Hoboken, New York, Piers",Unknown,Explosion,38.820809,-96.331617,30/06/1900,30/06/1900
2,1900-0006-JAM,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),Unknown,JAM,Jamaica,Latin America and the Caribbean,Americas,Saint James,Unknown,Unknown,15.100000,36.720000,06/01/1900,06/01/1900
3,1900-0007-JAM,nat-bio-epi-vir,Natural,Biological,Epidemic,Viral disease,Gastroenteritis,JAM,Jamaica,Latin America and the Caribbean,Americas,Porus,Unknown,Unknown,18.122079,-77.303589,13/01/1900,13/01/1900
4,1900-0008-JPN,nat-geo-vol-ash,Natural,Geophysical,Volcanic activity,Ash fall,Unknown,JPN,Japan,Eastern Asia,Asia,Unknown,Unknown,Unknown,3.170000,22.102000,07/07/1900,07/07/1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,2024-9277-VNM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,VNM,Vietnam,South-eastern Asia,Asia,"Can Duoc, Can Giuoc areas (Long An province); ...",Unknown,Unknown,16.517347,105.913388,01/01/2024,31/06/2024
26910,2024-9338-NAM,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,NAM,Namibia,Sub-Saharan Africa,Africa,"Khomas, Ohangwena, Kavango West, Kunene, and K...",El Nino,Food shortage|Water shortage,-21.908582,18.164513,01/05/2024,31/09/2024
26911,2024-9381-GRD,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,GRD,Grenada,Latin America and the Caribbean,Americas,"Saint Andrew, Saint David, Saint George",Unknown,Water shortage,12.112927,-61.679379,01/05/2024,31/07/2024
26912,2024-9592-AGO,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,Unknown,AGO,Angola,Sub-Saharan Africa,Africa,Unknown,El Nino,Food shortage,-12.167424,17.651769,01/06/2024,31/08/2024


continuedd

In [550]:
# Save the cleaned dataset
df_merged.to_csv("cleaned_natural_disasters.csv", index=False)